# 定义无标签数据集类

In [1]:
import numpy as np 
import os 
from torch.utils.data import Dataset, DataLoader

class UnlabeledMelFrequencyDataset(Dataset):
    def __init__(self, data_dir, max_seq_length=300):
        self.data = []
        self.max_seq_length = max_seq_length
        data_files = sorted(os.listdir(data_dir))  # 对文件列表进行排序
        for file in data_files:
            file_path = os.path.join(data_dir, file)
            self.data.append(file_path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        file_path = self.data[idx]
        # 加载无标签梅尔频率数据
        mel_data = np.load(file_path)  # 假设是.npy格式的文件
        # 对数据进行填充或截断
        if mel_data.shape[0] < self.max_seq_length:
            # 填充到最大长度
            pad_length = self.max_seq_length - mel_data.shape[0]
            mel_data = np.pad(mel_data, ((0, pad_length), (0, 0)), mode='constant', constant_values=0)
        elif mel_data.shape[0] > self.max_seq_length:
            # 截断到最大长度
            mel_data = mel_data[:self.max_seq_length, :]
        # 将梅尔频率特征转换为PyTorch张量并返回
        return torch.tensor(mel_data)

# 载入预测数据集

In [2]:
# 创建无标签数据集对象
unlabeled_dataset = UnlabeledMelFrequencyDataset(data_dir='data/test_data', max_seq_length=300)
# 创建数据加载器
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=64, shuffle=False)

# 确认按文件顺序排列
# 打印出前 10 个文件路径
for i in range(20):
    print(unlabeled_dataset.data[i])

data/test_data/test_0001.npy
data/test_data/test_0002.npy
data/test_data/test_0003.npy
data/test_data/test_0004.npy
data/test_data/test_0005.npy
data/test_data/test_0006.npy
data/test_data/test_0007.npy
data/test_data/test_0008.npy
data/test_data/test_0009.npy
data/test_data/test_0010.npy
data/test_data/test_0011.npy
data/test_data/test_0012.npy
data/test_data/test_0013.npy
data/test_data/test_0014.npy
data/test_data/test_0015.npy
data/test_data/test_0016.npy
data/test_data/test_0017.npy
data/test_data/test_0018.npy
data/test_data/test_0019.npy
data/test_data/test_0020.npy


# 完成预测任务

In [3]:
import torch
import models.network as models
from torch.utils.data import Dataset, DataLoader

# 检查CUDA是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定义模型结构
model = models.MelTransformer().to(device)
# 加载已保存的模型参数
model.load_state_dict(torch.load('saved_models/model_tfm_v4.pth'))
# 将模型设置为评估模式
model.eval()

# 使用模型进行预测
predictions = []
for inputs in unlabeled_loader:
    inputs = inputs.to(device)  # 将输入数据移动到设备上
    outputs = model(inputs)
    _, predicted = torch.max(outputs, 1)  # 获取预测结果
    predictions.extend(predicted.tolist())

# 打印预测结果
print(predictions)

[1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 